In [72]:
import pandas as pd
from scipy.stats import combine_pvalues

In [73]:
BanSun = pd.read_excel("Example_DEresult1.xlsx")
MtSin = pd.read_excel("Example_DEresult3.xlsx")
Emory = pd.read_excel("Example_DEresult2.xlsx")

In [74]:
combine_df = BanSun.merge(MtSin, on = "Identifier", how="inner").merge(Emory, on = "Identifier", how="inner")
combine_df = combine_df[~(combine_df.Identifier.str.contains("co.")|combine_df.Identifier.str.contains("cu."))]
combine_df = combine_df.set_index("Identifier")
gene = combine_df[['GN']]

In [75]:
df = combine_df.iloc[:,combine_df.columns.str.contains("AD")& ~combine_df.columns.str.contains("AsymAD")& ~combine_df.columns.str.contains("adj.P.Val")]

In [76]:
df.columns = ['AD-Ctl_logFC_BanSun', 'AD-Ctl_P.Value_BanSun', 'AD-Ctl_logFC_MtSinai', 'AD-Ctl_P.Value_MtSinai',
       'AD-Ctl_logFC_Emory', 'AD-Ctl_P.Value_Emory']

In [78]:
# Convert two-tailed p-value to one-tailed p-value for positive trend
def two_to_one_tailed_pos(p_value, logFC):
    if logFC > 0:
        return p_value / 2
    else:
        return (1 - p_value / 2)

# Convert two-tailed p-value to one-tailed p-value for negative trend
def two_to_one_tailed_neg(p_value, logFC):
    if logFC < 0:
        return p_value / 2
    else:
        return (1 - p_value / 2)

# Function to apply the appropriate one-tailed conversion based on FC trend
def apply_conversion(row, p_value_col, logFC_col):
    if row['FCtrend'] > 0:
        return two_to_one_tailed_pos(row[p_value_col], row[logFC_col])
    else:
        return two_to_one_tailed_neg(row[p_value_col], row[logFC_col])

# Combine p-values using Fisher's method
def combine_pvalues_fisher(pvalues):
    return combine_pvalues(pvalues, method='fisher')[1]

In [79]:
# Calculate the mean logFC trend for each row
df['FCtrend'] = df.loc[:, df.columns.str.contains("logFC")].mean(axis=1)

# Apply the appropriate one-tailed conversion function for each p-value column
df['One_Tailed_P.Value_BanSun'] = df.apply(lambda row: apply_conversion(row, 'AD-Ctl_P.Value_BanSun', 'AD-Ctl_logFC_BanSun'), axis=1)
df['One_Tailed_P.Value_MtSinai'] = df.apply(lambda row: apply_conversion(row, 'AD-Ctl_P.Value_MtSinai', 'AD-Ctl_logFC_MtSinai'), axis=1)
df['One_Tailed_P.Value_Emory'] = df.apply(lambda row: apply_conversion(row, 'AD-Ctl_P.Value_Emory', 'AD-Ctl_logFC_Emory'), axis=1)


# Combine the one-tailed p-values
df['Combined_P.Value'] = df.apply(lambda row: combine_pvalues_fisher([
    row['One_Tailed_P.Value_BanSun'], 
    row['One_Tailed_P.Value_MtSinai'], 
    row['One_Tailed_P.Value_Emory']]), axis=1)


C:\Users\hshresth\AppData\Local\Temp\ipykernel_19064\1694680959.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['FCtrend'] = df.loc[:, df.columns.str.contains("logFC")].mean(axis=1)
C:\Users\hshresth\AppData\Local\Temp\ipykernel_19064\1694680959.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['One_Tailed_P.Value_BanSun'] = df.apply(lambda row: apply_conversion(row, 'AD-Ctl_P.Value_BanSun', 'AD-Ctl_logFC_BanSun'), axis=1)
C:\Users\hshresth\AppData\Local\Temp\ipykernel_19064\1694680959.py:6: Set

In [80]:
import statsmodels.stats.multitest as smm

In [81]:
df['BH_Corrected_P.Value'] = smm.multipletests(df['Combined_P.Value'], method='fdr_bh')[1]

C:\Users\hshresth\AppData\Local\Temp\ipykernel_19064\2550239702.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BH_Corrected_P.Value'] = smm.multipletests(df['Combined_P.Value'], method='fdr_bh')[1]


In [89]:
df = gene.merge(df, left_index=True, right_index=True, how="outer")

In [90]:
df.to_csv("Combined_DE_AD_cohorts.csv")